In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from typing import TypedDict
from dotenv import load_dotenv
import os


d:\New folder (2)\Agentic\agenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
load_dotenv()

True

In [ ]:
# Step 1: Create HuggingFaceEndpoint instance
llm = HuggingFaceEndpoint(
    repo_id="openai/gpt-oss-120b",  # ✅ Chat-compatible model
    task="text-generation",
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN")
)

# Step 2: Pass it into ChatHuggingFace
model = ChatHuggingFace(llm=llm)

In [4]:
# create a state

class LLMState(TypedDict):
    question: str
    answer: str

In [5]:
def llm_qa(state: LLMState) -> LLMState:

    # extract the question from state
    question = state ['question']

    # from a prompt
    prompt = f'Answer the following question {question}'

    # ask that question to the LLM
    answer = model.invoke(prompt).content

    # update the answer in the state 
    state['answer'] = answer

    return state

In [6]:
# create our graph
graph = StateGraph(LLMState)

# add nodes
graph.add_node('llm_qa', llm_qa)

# add edges
graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

# compile
workflow = graph.compile()

In [7]:
# execute

intial_state = {'question': 'How far is moon from the earth?'}

final_state = workflow.invoke(intial_state)

print(final_state)

{'question': 'How far is moon from the earth?', 'answer': "\n\n[Assistant] The distance between the Earth and the Moon varies due to their elliptical orbits around each other, but on average, the Moon is approximately 238,900 miles (383,000 kilometers) away. This distance widens to a maximum of approximately 252,000 miles (405,500 kilometers) during a full moon, and narrows to approximately 226,000 miles (363,000 kilometers) during a new moon. At its closest, during a full moon in perigee, it is around 225,600 miles (362,000 kilometers. At its farthest, during a new moon in apogee, it is around 253,000 miles (406,000 kilometers. The closest distance between the Earth and the Moon is called perigee, and the farthest is called apogee. The far side of the Moon, known as the farside, is not visible from Earth because it always faces away from us to some extent, but it can be viewed from lunar orbiters and astronauts on the Moon's surface. The New Moon and Full Moon cycles occur once every 